<a href="https://colab.research.google.com/github/DanielGreiner/Masterarbeit/blob/main/Isothermal_steady_state_PFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libraries

In [1]:
# @title import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import torch as to
from scipy.integrate import odeint

In [2]:
# @title check library versions
print("tensorflow:" + str(tf.__version__))
print("numpy:" + str(np.__version__))
print("pandas:" + str(pd.__version__))
#print("torch:" + str(to.__version__))

tensorflow:2.13.0
numpy:1.23.5
pandas:1.5.3


# Reaktionen

  \begin{array}{lll}
  \text{Methanol steam reforming (MSR):}&\kern 3pc CH_3OH_{(g)}+H_2O_{(g)}\kern 0.5pc{\overset{k_{1, eff}}{\rightleftharpoons}}\kern 0.5pc 3H_{2(g)}+CO_{2(g)}&\kern 3pc\Delta_r H_{m}^{o}=+49\enspace \frac{kJ}{mol} \\
  \text{Methanol decomposition (MD):}&\kern 3pc CH_3OH_{(g)}\kern 0.5pc {\overset{k_{2, eff}}{\rightleftharpoons}}\kern 0.5pc 2H_{2(g)}+CO_{(g)}&\kern 3pc\Delta_r H_{m}^{o}=+91\enspace \frac{kJ}{mol} \\
  \text{Water gas shift (WGS):}&\kern 3pc CO_{(g)}+H_2O_{(g)}\kern 0.5pc{\overset{k_{3, eff}}{\rightleftharpoons}}\kern 0.5pc H_{2(g)}+CO_{2(g)}&\kern 3pc\Delta_r H_{m}^{o}=-41\enspace \frac{kJ}{mol}
  \end{array}


#Indexing in arrays

\begin{array}{lllll}
H_2 & H_2O & CO & CO_2 & CH_3OH \\
0 & 1 & 2 & 3 & 4 \\
\end{array}

In [37]:
# @title Reaktorkonstanten laden
u = 1.500
A = 3*10**-3
v = u*A
Vges = 0.015
steps = 1000
dV = Vges/steps
z = Vges/A
r = np.sqrt(A/np.pi)
T0 = 273

0.003
0.003


In [31]:
# @title Startwerte
t0 = 0
z0 = 0
V0 = 0
T0 = 298
C0_H2 = 0
C0_H2O = 2
C0_CO = 0
C0_CO2 = 0
C0_MeOH = 2

IC = np.array([C0_H2, C0_H2O, C0_CO, C0_CO2, C0_MeOH])

In [35]:
# @title Kinetikdaten laden

R = 8.314 #J mol-1 K-1

#BiCat
#MSR
k0_MSR_S = 4.2*10**(-6) # kmol kg-1 h-1
EA_MSR_S = 4.978 #kJ mol-1
b_MeOH_MSR_S = - 0.428
b_H2O_MSR_S = - 0.949

k0_MSR_M = 2.1*10**(-6) # kmol kg-1 h-1
k0_MSR_K = 2.7*10**(-6) # kmol kg-1 h-1
k0_MSR_L = 3.3*10**(-6) # kmol kg-1 h-1

EA_MSR_MKL = 5.566
b_MeOH_MSR_MKL = - 0.43
b_H2O_MSR_MKL = - 0.949

MSR = np.array(
    [[k0_MSR_S, EA_MSR_S, b_MeOH_MSR_S, b_H2O_MSR_S],
    [k0_MSR_M, EA_MSR_MKL, b_MeOH_MSR_MKL, b_H2O_MSR_MKL],
    [k0_MSR_K, EA_MSR_MKL, b_MeOH_MSR_MKL, b_H2O_MSR_MKL],
    [k0_MSR_L, EA_MSR_MKL, b_MeOH_MSR_MKL, b_H2O_MSR_MKL]]
)

#MD
k0_MD_S = 5.5*10**(-8) # kmol kg-1 h-1
EA_MD_S = 42.137 #kJ mol-1
b_MeOH_MD_S = 0.114

k0_MD_M = 1.5*10**(-8) # kmol kg-1 h-1
k0_MD_K = 3.7*10**(-8) # kmol kg-1 h-1
k0_MD_L = 1.1*10**(-7) # kmol kg-1 h-1

EA_MD_MKL = 36.771 #kJ mol-1
b_MeOH_MD_MKL = 0.109

MD = np.array (
    [[k0_MD_S, EA_MD_S, b_MeOH_MD_S],
    [k0_MD_M, EA_MD_MKL, b_MeOH_MD_MKL],
    [k0_MD_K, EA_MD_MKL, b_MeOH_MD_MKL],
    [k0_MD_L, EA_MD_MKL, b_MeOH_MD_MKL]]
)

#WGS
#Benchmark = B; Benchmark-CuCl = BCC; Benchmark-MB = BMB


k0_WGS_B = 3.1*10**(-10)
EA_B = 104.4 #kJ mol-1
b_CO_WGS_B = - 0.892
b_H2O_WGS_B = - 0.135


k0_WGS_BCC = 1.6*10**(-9)
EA_BCC = 105.5 #kJ mol-1
b_CO_WGS_BCC = - 0.836
b_H2O_WGS_BCC = - 0.338

k0_WGS_BMB = 2.82*10**(-10)
EA_BMB = 78.6 #kJ mol-1
b_CO_WGS_BMB = - 0.685
b_H2O_WGS_BMB = - 0.268

WGS = np.array(
    [[k0_WGS_B, EA_B, b_CO_WGS_B, b_H2O_WGS_B],
    [k0_WGS_BCC, EA_BCC, b_CO_WGS_BCC, b_H2O_WGS_BCC],
    [k0_WGS_BMB, EA_BMB, b_CO_WGS_BMB, b_H2O_WGS_BMB]]
)

ny = np.array(
    [[3, -1, 0, 1, -1],
    [2, 0, 1, 0, -1],
    [1, -1, -1, 1, 0]]
)


#print(MSR)
#print(MD)
#print(WGS)
print(ny)

[[ 3 -1  0  1 -1]
 [ 2  0  1  0 -1]
 [ 1 -1 -1  1  0]]


# PFTR Isotherm Stationär

ODE: $\frac{dc_x}{dz}$=$\frac{1}{u_0}$⋅$R_x$

$\frac{dc_A}{dV}$=-$\frac{1}{V.}$⋅$R_x$

In [ ]:
# @title Funktion der Reaktionen
def ReakFun(t,C,T):
  dCdt=np.zeros(len(C))
  C_H2 = C[1]
  C_H2O = C[2]
  C_CO = C[3]
  C_CO2 = C[4]
  C_MeOH = C[5]

  rMSR = np.zeros(len(MSR))
  rMD = np.zeros(len(MD))
  rWGS = np.zeros(len(WGS))

  for i in range(len(MSR)):
    rMSR[i] = MSR[i,0]*np.exp((-MSR[i,1]/R)*(1/T-1/T0))*C[5]**MSR[i,2]*C[2]**MSR[i,3]


  for i in range(len(MD)):
    rMD[i] = MD[i,0]**np.exp((-MD[i,1]/R)*(1/T-1/T0))*C[5]**MSR[i,2]


  for i in range(len(WGS)):
    rWGS[i] = WGS[i,0]*np.exp((-WGS[i,1]/R)*(1/T-1/T0))*C[3]**WGS[i,2]*C[2]**WGS[i,3]

  dCdT[0] =
  dCdT[1] =
  dCdT[2] =
  dCdT[3] =
  dCdT[4] =
  dCdT[5] =


In [ ]:
# @title Euler Verfahren manuell
V = V0
cA = cA0
cAtemp = cA0
res = np.zeros((steps+1, 2))
res[0,0] = V0
res[0,1] = cA0

for i in range(steps):
  cA = cAtemp - (1/v)*k*cAtemp*dV
  cAtemp = cA
  V = V + dV
  res[i+1,0] = V*1000
  res[i+1,1] = cA

#print("cA = "+ str(cA))
#print("V = "+ str(V))
#print(res)

In [ ]:
# @title Berechnung mittels ODE-Solver

Vsp = np.arange(0, Vges, dV)

def dcdV(C, V):
  cA = C
  dCdV = -(1/v)*k*C
  return dCdV

sol = odeint(dcdV, cA0, Vsp)
#print(sol[:,0])
#print(Vsp)

In [ ]:
# @title Plots

plt.plot(res[:,0],res[:,1], 'b', label='Euler')
plt.plot(Vsp*1000, sol[:,0], 'r', label='Solver', linestyle='dashed')
plt.axis((0, 15, 0, 2))
plt.legend(loc='best')


In [46]:
# @title code testing block || IGNORE

H = np.array (
    [[1, 1, 1],
    [2, 2, 2],
    [7, 3, 3],
    [4, 4, 4]])

Hres = np.zeros(len(H))

for i in range(len(H)):
  Hres[i] = H[i,0]*H[i,1]*H[i,2]


#print(Hres)

A = 3*10**-3
B = 3*np.power(10,-3, dtype=float)

#print(A)
#print(B)

C=np.array(
    [[1, 2, 1, 1, 2],
    [1, 3, 1, 1, 3]]
)

rWGS = np.zeros(len(WGS))
rWGS2 = np.zeros(len(WGS))


for i in range(2):
  rWGS[i] = WGS[i,0]*np.exp((-WGS[i,1]/R)*(1/T-1/T0))*C[i, 3]**WGS[i,2]*C[i, 2]**WGS[i,3]
  rWGS2[i] = WGS[i,0]*np.exp((-WGS[i,1]/R)*(1/T-1/T0))*np.power(C[i, 3], WGS[i,2], dtype=float)*np.power(C[i, 2], WGS[i,3], dtype=float)

print(rWGS)
print(rWGS2)

[3.11285971e-10 1.60670735e-09 0.00000000e+00]
[3.11285971e-10 1.60670735e-09 0.00000000e+00]
